In [135]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 16: Ticket Translation ---</h2><p>As you're walking to yet another connecting flight, you realize that one of the legs of your re-routed trip coming up is on a high-speed train. However, the train ticket you were given is in a language you don't understand. You should probably figure out what it says before you get to the train station after the next flight.</p>
<p>Unfortunately, you <span title="This actually happened to me once, but I solved it by just asking someone.">can't actually <em>read</em> the words on the ticket</span>. You can, however, read the numbers, and so you figure out <em>the fields these tickets must have</em> and <em>the valid ranges</em> for values in those fields.</p>
<p>You collect the <em>rules for ticket fields</em>, the <em>numbers on your ticket</em>, and the <em>numbers on other nearby tickets</em> for the same train service (via the airport security cameras) together into a single document you can reference (your puzzle input).</p>
<p>The <em>rules for ticket fields</em> specify a list of fields that exist <em>somewhere</em> on the ticket and the <em>valid ranges of values</em> for each field. For example, a rule like <code>class: 1-3 or 5-7</code> means that one of the fields in every ticket is named <code>class</code> and can be any value in the ranges <code>1-3</code> or <code>5-7</code> (inclusive, such that <code>3</code> and <code>5</code> are both valid in this field, but <code>4</code> is not).</p>
<p>Each ticket is represented by a single line of comma-separated values. The values are the numbers on the ticket in the order they appear; every ticket has the same format. For example, consider this ticket:</p>
<pre><code>.--------------------------------------------------------.
| ????: 101    ?????: 102   ??????????: 103     ???: 104 |
|                                                        |
| ??: 301  ??: 302             ???????: 303      ??????? |
| ??: 401  ??: 402           ???? ????: 403    ????????? |
'--------------------------------------------------------'
</code></pre>
<p>Here, <code>?</code> represents text in a language you don't understand. This ticket might be represented as <code>101,102,103,104,301,302,303,401,402,403</code>; of course, the actual train tickets you're looking at are <em>much</em> more complicated. In any case, you've extracted just the numbers in such a way that the first number is always the same specific field, the second number is always a different specific field, and so on - you just don't know what each position actually means!</p>
<p>Start by determining which tickets are <em>completely invalid</em>; these are tickets that contain values which <em>aren't valid for any field</em>. Ignore <em>your ticket</em> for now.</p>
<p>For example, suppose you have the following notes:</p>
<pre><code>class: 1-3 or 5-7
row: 6-11 or 33-44
seat: 13-40 or 45-50

your ticket:
7,1,14

nearby tickets:
7,3,47
40,<em>4</em>,50
<em>55</em>,2,20
38,6,<em>12</em>
</code></pre>

<p>It doesn't matter which position corresponds to which field; you can identify invalid <em>nearby tickets</em> by considering only whether tickets contain <em>values that are not valid for any field</em>. In this example, the values on the first <em>nearby ticket</em> are all valid for at least one field. This is not true of the other three <em>nearby tickets</em>: the values <code>4</code>, <code>55</code>, and <code>12</code> are are not valid for any field. Adding together all of the invalid values produces your <em>ticket scanning error rate</em>: <code>4 + 55 + 12</code> = <em><code>71</code></em>.</p>
<p>Consider the validity of the <em>nearby tickets</em> you scanned. <em>What is your ticket scanning error rate?</em></p>
</article>


In [136]:
example = """
class: 1-3 or 5-7
row: 6-11 or 33-44
seat: 13-40 or 45-50

your ticket:
7,1,14

nearby tickets:
7,3,47
40,4,50
55,2,20
38,6,12
"""


def ticket_scanning_error_rate(s: str) -> int:
    rules, ticket, nearby = re.split(r"\n\s*\n", s.strip())
    rules = [
        range(*[int(n) + (1 if i else 0) for i, n in enumerate(r.split("-"))])
        for rule in rules.splitlines()
        for r in re.sub(r"^.+: ", "", rule).split(" or ")
    ]

    ticket = [int(n) for n in ticket.splitlines()[1].strip().split(",")]
    nearby = [[int(n) for n in t.split(",")] for t in nearby.splitlines()[1:]]

    rate = sum(n for n in ticket if all(n not in rule for rule in rules))
    rate += sum(n for t in nearby for n in t if all(n not in rule for rule in rules))
    return rate


assert ticket_scanning_error_rate(example) == 71

In [137]:
with open("../input/day16.txt") as f:
    puzzle = f.read()

print(f"Part I: {ticket_scanning_error_rate(puzzle)}")

Part I: 23925


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>23925</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>Now that you've identified which tickets contain invalid values, <em>discard those tickets entirely</em>. Use the remaining valid tickets to determine which field is which.</p>
<p>Using the valid ranges for each field, determine what order the fields appear on the tickets. The order is consistent between all tickets: if <code>seat</code> is the third field, it is the third field on every ticket, including <em>your ticket</em>.</p>
<p>For example, suppose you have the following notes:</p>
<pre><code>class: 0-1 or 4-19
row: 0-5 or 8-19
seat: 0-13 or 16-19

your ticket:
11,12,13

nearby tickets:
3,9,18
15,1,5
5,14,9
</code></pre>

<p>Based on the <em>nearby tickets</em> in the above example, the first position must be <code>row</code>, the second position must be <code>class</code>, and the third position must be <code>seat</code>; you can conclude that in <em>your ticket</em>, <code>class</code> is <code>12</code>, <code>row</code> is <code>11</code>, and <code>seat</code> is <code>13</code>.</p>
<p>Once you work out which field is which, look for the six fields on <em>your ticket</em> that start with the word <code>departure</code>. <em>What do you get if you multiply those six values together?</em></p>
</article>

</main>


In [138]:
from pprint import pprint

from tabulate import tabulate


example1 = """
class: 0-1 or 4-19
row: 0-5 or 8-19
seat: 0-13 or 16-19

your ticket:
11,12,13

nearby tickets:
3,9,18
15,1,5
5,14,9
"""


def valid(t: list[int], rules: dict[str, tuple[range, range]]):
    return all(any(v in rs[0] or v in rs[1] for rs in rules.values()) for v in t)


def assign_fields_to_columns(nr_fields, col_possible_fields):
    col_field = [None] * nr_fields
    assigned_fields = set()

    for col, fields in col_possible_fields:
        for field in fields:
            if field not in assigned_fields:
                col_field[col] = field
                assigned_fields.add(field)
                break
    return col_field


def determine_possible_fields_per_column(ticket, nearby, fields):
    valids = [t for t in nearby if valid(t, fields)]

    nr_valids = len(valids)
    nr_fields = len(ticket)

    col_possible_fields = [[] for _ in range(nr_fields)]
    for col in range(nr_fields):
        for field, (range1, range2) in fields.items():
            count = sum(1 for t in valids if t[col] in range1 or t[col] in range2)
            if count == nr_valids:
                col_possible_fields[col].append(field)

    col_possible_fields = sorted(
        enumerate(col_possible_fields), key=lambda t: len(t[1])
    )

    return nr_fields, col_possible_fields


def parse(s):
    rules_str, ticket, nearby = re.split(r"\n\s*\n", s.strip())

    ticket = [int(n) for n in ticket.splitlines()[1].strip().split(",")]
    nearby = [[int(n) for n in t.split(",")] for t in nearby.splitlines()[1:]]

    fields = {}

    for r in rules_str.splitlines():
        field, tmp = r.split(": ")
        fr_1, to_1, fr_2, to_2 = (int(i) for i in re.findall(r"\d+", tmp))
        fields[field] = range(fr_1, to_1 + 1), range(fr_2, to_2 + 1)
    return ticket, nearby, fields


def get_fields(s: str):
    ticket, nearby, fields = parse(s)

    nr_fields, col_possible_fields = determine_possible_fields_per_column(
        ticket, nearby, fields
    )

    col_field = assign_fields_to_columns(nr_fields, col_possible_fields)

    return [(f, ticket[i]) for i, f in enumerate(col_field)]


assert get_fields(example1), [("row", 11) == ("class", 12), ("seat", 13)]

In [139]:
from math import prod


print(
    f"\nPart II: {prod(v for f, v in get_fields(puzzle) if f and f.startswith('departure'))}"
)


Part II: 964373157673


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>964373157673</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
